# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda x, fn: lambda y: fn(f(y, x)), lambda x: x, lst)(x0)

In [6]:
foldl2(f, 1, [1, 2, 3])

0.16666666666666666

In [7]:
def foldr2(f, x0, lst):
    return foldl(lambda fn, x: lambda y: fn(f(x, y)), lambda x: x, lst)(x0)

In [8]:
foldr2(f, 1, [1, 2, 3])

1.5

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [9]:
def check_inv(a, b):
    assert len(a) >= len(b)
    
    newA = dict.fromkeys(a, 0)
    newB = dict.fromkeys(a, 0)
    
    for char in b:
        newB[char] += 1
        
    for char in a[:len(b) - 1]:
        newA[char] += 1
        
    for i in range(len(b) - 1, len(a)):
        newA[a[i]] += 1
        
        if (newA == newB):
            return True
        
        newA[a[i - len(b) + 1]] -= 1
        
    return False

In [10]:
check_inv('adcrotm', 'crad')

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [11]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if(self.left == None and self.right == None):
            yield self.value
            
        if(self.left):
            yield from self.left
            
        if(self.right):
            yield from self.right
        
    
    def __str__(self):
        result = [str(self.value)]
        
        if(self.left):
            result.append(self.left.__str__())
            
        if(self.right):
            result.append(self.right.__str__())
            
        return ' '.join(res)
    
    def __repr__(self):
        if(self.left == None and self.right == None):
            return f"Tree({self.value})"
        
        if(self.right == None):
            return f"Tree({self.value}, {self.left.__repr__()})"
        
        if(self.left == None):
            return f"Tree({self.value}, None, {self.right.__repr__()})"
        
        
        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
 
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [12]:
import operator

def compute(expr):
    ops = {'-': operator.sub, '+': operator.add, '*': operator.mul, '/':operator.truediv}
    i = 0
    symbols = []
    for i in expr:
        if i.isalnum():
            symbols.append(i)
        else:
            s1 = int(symbols.pop())
            s2 = int(symbols.pop())
            op = ops[i]
            symbols.append(op(s2, s1))
    return symbols.pop()

def calc(expr):
    expr = expr.replace('(', '( ').replace(')', ' )').split()
    nums = []
    ops = []
    i = 0
    priority = {'-':0, '+':0, '*':1, '/':1, '(':0, ')':0}
    for t in expr:
        if t.isalnum():
            nums.append(t)
        elif t in ['+', '-', '*', '/']:
            while len(ops) > 0 and ops[-1] != '(' and (priority[t] < priority[ops[-1]] or priority[t] == priority[ops[-1]]):
                nums.append(ops.pop())
            ops.append(t)
        elif t == '(':
            ops.append(t)
        elif t == ')':
            while ops[-1] != '(':
                nums.append(ops.pop())
            if ops[-1] == '(':
                ops.pop()
    nums.extend(ops)
    return compute(nums)

In [13]:
calc('1 + 2') == 4

False

In [14]:
calc('2 * (15 - 3 * 4) - 2') == 4

True